# Algorithme de trading

Dans cette dernière partie du projet, nous récupérons les deux modèles entraînés ainsi que les articles de presse, les posts Reddit et le cours de l'action Apple pour conseiller à notre utilisateur d'acheter ou non une action Apple avant que le marché n'ouvre.

### Renseignez la date du jour

In [1]:
from datetime import datetime, timedelta


#On demande à l'utilisateur de saisir une date

user_date_input = input("Entrez la date d'aujourd'hui (format AAAA-MM-JJ): ")
current_date = datetime.strptime(user_date_input, "%Y-%m-%d")

On rappelle les fonctions utiles, codées précédemment :

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
from bs4 import BeautifulSoup

#Nettoyage du texte
def cleaning_text(text):
    #Passage du texte en miniscules
    text=text.lower()
    #Suppression des chiffres
    text=re.sub(r'\d+', '', text)
    #Suppression de la ponctuation et des symboles spéciaux
    text=re.sub(r'[^\w\s]', '', text)
    return text


stops = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

#Suppression des stopwords
def clean_stopwords(text):
    words = word_tokenize(text)
    cleaned_text=[word for word in words if word not in stops]
    return ' '.join(cleaned_text)

#Fonction de nettoyage du contenu html adaptée à la structure des articles Forbes
def clean_html_1(text_html): 
    soup=BeautifulSoup(text_html, 'html.parser')
    title=soup.find_all('h1', class_=True)
    content=soup.find_all('p')
    united_content = ' '.join(el.get_text(strip=True) for el in title + content )
    return united_content

#Fonctions de récupérations des commentaires principaux
def extract_submission_id(url):
    match = re.search(r'/comments/(\w+)/', url)
    return match.group(1) if match else None

def get_top_comments(url):
    submission_id = extract_submission_id(url)
    if not submission_id:
        print(f"Erreur pour '{url}'")
        return []
    submission = reddit.submission(id=submission_id)
    submission.comment_sort = 'top'
    submission.comments.replace_more(limit=0)
    
    top_comments = []
    for comment in submission.comments[:5]:  # Prendre les 5 premiers commentaires
        top_comments.append(comment.body)
    
    return top_comments


## Récupération des articles de presse de Forbes des 48h précédentes

On utilise à nouveau le code rédigé dans la partie PressArticles.ipynb en l'adaptant à la date rentrée par l'utilisateur. 

In [3]:
import requests

start_date = current_date - timedelta(days=2)
end_date = current_date
parameters = {
    'q': 'Apple',
    'domains': 'forbes.com',
    'from': start_date.strftime("%Y-%m-%d"),
    'to': end_date.strftime("%Y-%m-%d"),
    'sortBy': 'publishedAt',
    'apiKey': '1b20fb6f9b9d40f0b4e4ad6fe5d90755'  
}

url = 'https://newsapi.org/v2/everything'


response = requests.get(url, params=parameters)
articles = response.json().get('articles', [])  

/Users/dabadieraphael/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Nettoyage des articles

On stock les articles récupérés pour l'analyse de sentiments dans un **nouveau fichier CSV** afin de permettre à l'utilisateur d'y accéder facilement et de potentiellement accéder à son contenu.

In [4]:
import csv
import pandas as pd
import numpy as np

with open('articles_algo.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Titre', 'URL', 'Date de publication', 'Description','Source'])
    for article in articles:
        if "Apple" not in article['title']:
          continue
        writer.writerow([article['title'], article['url'], article['publishedAt'], article['description'], article['source']['name']])

df=pd.read_csv('articles_algo.csv')

urls = [url for url in df['URL']]
html_contents = []
compteur=0
df['Contenu']=[None] * len(df)

for url in urls:
    response = requests.get(url)
    if response.status_code == 200:
        html_contents.append(response.text)
        df['Contenu'].loc[compteur]=response.text
    else:
        print(f"Échec de récupération pour {url}")
    compteur+=1

df.head()

,Titre,URL,Date de publication,Description,Source,Contenu
0,"Apple Execs On Continuity, The Secret Sauce Th...",https://www.forbes.com/sites/davidphelan/2023/...,2023-12-29T14:00:57Z,Apple’s products talk to each other with an in...,Forbes,"<!DOCTYPE html><html lang=""en""><head><link rel..."
1,"Apple Watch Series 9, Ultra 2 Back On Sale Now...",https://www.forbes.com/sites/davidphelan/2023/...,2023-12-28T10:00:42Z,The latest twist in the Apple Watch sales ban ...,Forbes,"<!DOCTYPE html><html lang=""en""><head><link rel..."


In [5]:
if len(df)>0:
    df['Cleaned_html_content'] = df.apply(lambda row: clean_html_1(row['Contenu']), axis=1)
    df['Content_cleaned']=df['Cleaned_html_content'].apply(cleaning_text)
    df['Content_cleaned_from_stopwords']=df['Content_cleaned'].apply(clean_stopwords)

### Analyse de sentiments avec un modèle pré-entraîné

Comme vu dans PressArticles.ipynb, on utilise le modèle transformers qui est le plus performant pour analyser les articles récupérés. On associe à chacun d'entre eux un label "**POSITIVE**" ou "**NEGATIVE**".

In [6]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis')

sentiments=[]
#Comme nous l'avions vu précédemment, il faut tronquer les textes pour les réduire à 512 tokens au risque d'obtenir un message d'erreur
for i in range(len(df)):
    text = df.iloc[i,8]
    text = text[:512]
    sentiment = classifier(text)[0]['label']  
    sentiments.append(sentiment)
df['predicted_sentiment'] = sentiments
df.head()



/Users/dabadieraphael/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


,Titre,URL,Date de publication,Description,Source,Contenu,Cleaned_html_content,Content_cleaned,Content_cleaned_from_stopwords,predicted_sentiment
0,"Apple Execs On Continuity, The Secret Sauce Th...",https://www.forbes.com/sites/davidphelan/2023/...,2023-12-29T14:00:57Z,Apple’s products talk to each other with an in...,Forbes,"<!DOCTYPE html><html lang=""en""><head><link rel...","Apple Execs On Continuity, The Secret Sauce Th...",apple execs on continuity the secret sauce tha...,apple execs continuity secret sauce makes ipho...,POSITIVE
1,"Apple Watch Series 9, Ultra 2 Back On Sale Now...",https://www.forbes.com/sites/davidphelan/2023/...,2023-12-28T10:00:42Z,The latest twist in the Apple Watch sales ban ...,Forbes,"<!DOCTYPE html><html lang=""en""><head><link rel...","Apple Watch Series 9, Ultra 2 Back On Sale Now...",apple watch series ultra back on sale now in...,apple watch series ultra back sale latest surp...,NEGATIVE


On récupère le pourcentage d'articles à connotation positive parmi les articles qu'on a récupérés.

In [7]:
rate_pos = (df['predicted_sentiment'] == 'POSITIVE').mean()
percentage_pos = rate_pos * 100
print(f"Pourcentage de prédictions 'POSITIVE': {percentage_pos:}")


Pourcentage de prédictions 'POSITIVE': 50.0


## Récupération des posts Reddit

In [8]:
pip install praw

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import praw
import pandas as pd
from datetime import datetime, timedelta

reddit = praw.Reddit(client_id='xIq0ALkJ0RWzM5ZLwwiQKA',
                     client_secret='DeHliktGK8nfhDXsJFiebqgeZKhHXQ',
                     user_agent='Matlpg')

start_date = current_date - timedelta(days=2)

subreddit = reddit.subreddit('apple')
top_posts = subreddit.new(limit=500)  # On récupère ainsi les 500 derniers posts Reddit sur Apple

posts_data = []
for post in top_posts:
    post_date = datetime.utcfromtimestamp(post.created_utc)
    if start_date <= post_date < current_date:
        text = post.selftext if post.selftext else "Text Not Available"
        post_data = {
            "Titre": post.title,
            "Auteur": str(post.author),
            "Texte": text,
            "Date": post_date,
            "url": post.url
        }
        posts_data.append(post_data)

df_reddit = pd.DataFrame(posts_data)


In [10]:
df_reddit['Cleaned_text']=df_reddit['Texte'].apply(cleaning_text)
df_reddit['Cleaned_text']=df_reddit['Cleaned_text'].apply(clean_stopwords)
df_reddit = df_reddit[~df_reddit['Texte'].str.contains("Text Not Available")]
df_reddit.head()

,Titre,Auteur,Texte,Date,url,Cleaned_text
0,Microsoft Copilot app released for iOS,parity_bits,It’s basically just the Copilot tab from the B...,2023-12-29 22:30:29,https://apps.apple.com/us/app/microsoft-copilo...,basically copilot tab bing app without bing bl...
2,"Daily Advice Thread - December 29, 2023",AutoModerator,Welcome to the Daily Advice Thread for /r/Appl...,2023-12-29 11:00:34,https://www.reddit.com/r/apple/comments/18tkm9...,welcome daily advice thread rapple thread used...
3,Swipe on Dynamic Island (is this well known?) ...,safereddddditer175,\nIs this a well known feature? In MKBHD’s lat...,2023-12-29 10:08:57,https://youtu.be/YmwskGLycHo?si=oKM--J62UM7SE_hH,well known feature mkbhds latest iphone pro re...
6,"Daily Advice Thread - December 28, 2023",AutoModerator,Welcome to the Daily Advice Thread for /r/Appl...,2023-12-28 11:00:30,https://www.reddit.com/r/apple/comments/18srcx...,welcome daily advice thread rapple thread used...


On récupère les commentaires **les plus importants** :

In [11]:
for index, row in df_reddit.iterrows():
    top_comments = get_top_comments(row['url'])
    for i, comment in enumerate(top_comments):
        df_reddit.loc[index, f'comment_{i+1}'] = comment

Erreur pour 'https://apps.apple.com/us/app/microsoft-copilot/id6472538445'
Erreur pour 'https://youtu.be/YmwskGLycHo?si=oKM--J62UM7SE_hH'


In [12]:
df_reddit.replace('nan', np.nan, inplace=True) #Il y avait un problème avec le formatage précédent des 'nan'
df_reddit = df_reddit.dropna(axis=0, subset=['comment_1', 'comment_2', 'comment_3', 'comment_4', 'comment_5'])
df_reddit.head()

,Titre,Auteur,Texte,Date,url,Cleaned_text,comment_1,comment_2,comment_3,comment_4,comment_5
2,"Daily Advice Thread - December 29, 2023",AutoModerator,Welcome to the Daily Advice Thread for /r/Appl...,2023-12-29 11:00:34,https://www.reddit.com/r/apple/comments/18tkm9...,welcome daily advice thread rapple thread used...,I recently sent off my Apple Watch for repair ...,"Guys, I fucked up, made a factory reset on my ...",I have a M1 Macbook Air 13 that I love but it'...,I’m looking into buying a Studio Display to pa...,"If you invite people into Family Sharing, does..."
6,"Daily Advice Thread - December 28, 2023",AutoModerator,Welcome to the Daily Advice Thread for /r/Appl...,2023-12-28 11:00:30,https://www.reddit.com/r/apple/comments/18srcx...,welcome daily advice thread rapple thread used...,I just impulsively upgraded from an iPhone 12 ...,I am seriously considering purchasing two Appl...,App Store Always Requiring Password\n\nWhy is ...,I have a files app on my iPhone that no longer...,I've read a few threads regarding this and rea...


**Nettoyage** des commentaires :

In [13]:
df_reddit['comment_1_clean'] = df_reddit['comment_1'].apply(cleaning_text)
df_reddit['comment_2_clean'] = df_reddit['comment_2'].apply(cleaning_text)
df_reddit['comment_3_clean'] = df_reddit['comment_3'].apply(cleaning_text)
df_reddit['comment_4_clean'] = df_reddit['comment_4'].apply(cleaning_text)
df_reddit['comment_5_clean'] = df_reddit['comment_5'].apply(cleaning_text)

Analyse de **sentiments**, une fois de plus avec transformers :

In [22]:
model_transformers = pipeline("sentiment-analysis")
transformers_sentiments = []
for index, row in df_reddit.iterrows():
    for col in ['comment_1_clean', 'comment_2_clean', 'comment_3_clean', 'comment_4_clean', 'comment_5_clean']:
        transformers_result = model_transformers(row[col][:512])[0] #Il faut limiter la taille du texte à 512 caractères pour ce mmodèle comme vu précédemment
        transformers_sentiment = transformers_result['label'].lower()
        transformers_sentiments.append(transformers_sentiment)

transformers_sentiments

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


['negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative']

In [15]:
pos=0
for el in transformers_sentiment:
    if el=="positive":
        pos+=1
pos_reddit=(pos/len(df_reddit))*100
print(f"Il y a {pos_reddit}% de posts positifs")

Il y a 0.0% de posts positifs


## Récupération des informations de Yahoo Finance 

On l'a vu dans le fichier dédié.

In [16]:
import matplotlib.pyplot as plt 
import numpy as np
import yfinance as yf 
import pandas as pd 
from datetime import datetime

APL = "AAPL"
data = yf.Ticker(APL) # Extraction avec la fonction yf.Ticker de yfinance
prix_rec = data.history(period = '1d', start = '2020-1-1', end = current_date)
prix_rec.to_csv('AAPLt.csv')
df=pd.read_csv("AAPLt.csv") 
df['Date'] = df['Date'].astype(str)
df['Date'] = df['Date'].str.slice(0, 10)
df['Date'] = pd.to_datetime(df['Date'])
df = df[['Date', 'Close']]
df

,Date,Close
0,2020-01-02,73.152657
1,2020-01-03,72.441460
2,2020-01-06,73.018684
3,2020-01-07,72.675270
4,2020-01-08,73.844337
...,...,...
1001,2023-12-22,193.600006
1002,2023-12-26,193.050003
1003,2023-12-27,193.149994
1004,2023-12-28,193.580002


### Prédiction de la valeur du lendemain 

On a vu dans la partie dédiée que le modèle le plus efficace était le naive forecast.
Cependant, par définition du modèle, la valeur du lendemain que le modèle va prédire est la valeur réelle d'aujourd'hui. Donc il ne sera pas possible de tirer de conclusion sur oui ou non il faut acheter l'action, car sa valeur prédite sera la même qu'aujourd'hui.

Pour résoudre ce problème, nous allons comparer la valeur prédite, donc la valeur à J+1, avec la valeur à J-1.
Si la valeur est supérieure, alors nous considérerons qu'il faut acheter. 
Sinon, il vaut mieux ne pas acheter.
Une autre solution proposée est d'utiliser quand même le modèle LSTM, bien qu'il soit moins précis que le modèle de naive forecast, mais qui ne nécessite pas de recourir à ce compromis.

La valeur prédite du lendemain est donc la valeur de l'action au jour d'input, c'est à dire la dernière valeur de fermeture du marché :

In [17]:
valeur_lendemain = df['Close'].iloc[-1]
valeur_lendemain

192.52999877929688

Comparons-la à la valeur du jour J-1, qui est donc :

In [18]:
valeur_prec = df['Close'].iloc[-2]
valeur_prec

193.5800018310547

In [19]:
def acheter_ou_non():
    if valeur_lendemain > valeur_prec :
        return 'POSITIVE'
    else : 
        return 'NEGATIVE'
    
acheter_ou_non()

'NEGATIVE'

# Conclusion 

In [20]:
if rate_pos>0.5 and pos_reddit>50 and acheter_ou_non()=='POSITIVE':
    print("C'est le moment d'acheter")
else :
    print("Passez votre tour")


Passez votre tour


Nous avons fait le choix de ne recommander à notre utilisateur d'acheter l'action que si tous les voyants sont au vert mais nous pourrions facilement **changer les critères sélectionnés**. Nous avons montré dans ce projet qu'il était possible d'utiliser à la fois du **NLP** et de la **prédiction de séries temporelles** pour créer un algorithme de trading.

Avec **plus de ressources et de données**, nous aurions pu entraîner notre **propre modèle** sur des **données plus pertinentes** et ainsi l'aider à **généraliser** aux articles et posts traités ici. Ce projet pourrait être facilement augmenté en récupérant par exemple **massivement** des données de X (anciennement Twitter) et en mettant en place une exécution en **temps réel** de l'algorithme. 

Ce projet sera amené à être amélioré et testé sur des données historiques.